In [ ]:
# Load all necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the video statistics dataset
df = pd.read_csv("videos-stats.csv")

In [ ]:
# Clean and prepare the data for analysis
df['Published At'] = pd.to_datetime(df['Published At'], errors='coerce')
df.dropna(subset=['Views', 'Likes', 'Comments'], inplace=True)
df['Views'] = pd.to_numeric(df['Views'], errors='coerce')
df['Likes'] = pd.to_numeric(df['Likes'], errors='coerce')
df['Comments'] = pd.to_numeric(df['Comments'], errors='coerce')
df.drop_duplicates(inplace=True)
df['like_view_ratio'] = df['Likes'] / df['Views']

In [ ]:
# Store the DataFrame in an in-memory SQLite database for SQL queries
conn = sqlite3.connect(':memory:')
df.to_sql("youtube_stats", conn, index=False, if_exists="replace")

In [ ]:
# Find the 10 most viewed videos
query_top_views = """
SELECT Title, Views, Likes, Comments, like_view_ratio
FROM youtube_stats
ORDER BY Views DESC
LIMIT 10
"""
top_views = pd.read_sql_query(query_top_views, conn)
print("Top 10 Videos by View Count:")
print(top_views)

In [ ]:
# Identify videos with the highest like-to-view ratio
query_top_ratio = """
SELECT Title, Views, Likes, like_view_ratio
FROM youtube_stats
WHERE Views > 100000
ORDER BY like_view_ratio DESC
LIMIT 10
"""
top_ratios = pd.read_sql_query(query_top_ratio, conn)
print("\nTop 10 Videos by Like-to-View Ratio:")
print(top_ratios)

In [ ]:
# Plot how like-to-view ratios are distributed
plt.figure(figsize=(10, 6))
sns.histplot(df['like_view_ratio'], bins=50, kde=True)
plt.title('Distribution of Like-to-View Ratios')
plt.xlabel('Like/View Ratio')
plt.ylabel('Frequency')
plt.show()